In [ ]:
import ray
import time

In [ ]:
if ray.is_initialized():
    ray.shutdown()

ray.init()

In [ ]:
@ray.remote
def task():
    time.sleep(60)


@ray.remote
class Actor:
    def call(self):
        print("Actor called.")

In [ ]:
task.remote()

In [ ]:
!ray summary tasks

In [ ]:
actor = Actor.remote()

In [ ]:
actor.call.remote()

In [ ]:
!ray list actors

In [ ]:
### YOUR CODE HERE ###

In [ ]:
!ray get a913f6c55fe74f67bdf5f1b701000000

In [ ]:
### YOUR CODE HERE ###

In [ ]:
task.remote()
sample_actor = Actor.remote()
sample_actor.call.remote()

In [ ]:
if ray.is_initialized():
    ray.shutdown()

ray.init()

In [ ]:
running = False  # Set to True to run the memory leaker.


@ray.remote(max_retries=0)
def memory_leaker():
    chunks = []
    bytes_per_chunk = 1024 * 1024 * 1024  # 1 gigabyte.
    while running:
        chunks.append([0] * bytes_per_chunk)
        time.sleep(5)  # Delay to observe the leak.


ray.get(memory_leaker.remote())

In [ ]:
if ray.is_initialized():
    ray.shutdown()

ray.init()

In [ ]:
### SAMPLE STARTER SCRIPT ###


@ray.remote
class Leaker:
    def __init__(self):
        self.leaks = []

    def allocate(self, num_bytes: int, sleep_time_s: int):
        # Each element in the array occupies 8 bytes.
        new_list = [0] * ceil(num_bytes / 8)
        self.leaks.append(new_list)

        time.sleep(sleep_time_s)


### YOUR CODE HERE ###

In [ ]:
import random

In [ ]:
if ray.is_initialized():
    ray.shutdown()

ray.init()

In [ ]:
@ray.remote
def long_running_task():
    time.sleep(random.randint(10, 60))

@ray.remote
def dependent_task(dependencies: list[ray._raylet.ObjectRef]):
    ray.get(dependencies)

dependencies = [long_running_task.remote() for _ in range(100)]
dependent_task.remote(dependencies)

In [ ]:
if ray.is_initialized():
    ray.shutdown()

ray.init()

In [ ]:
@ray.remote
def sleep_task(i: int) -> int:
    time.sleep(i)
    return i


def post_processing_step(new_val: int):
    time.sleep(0.5)


big_sleep_times = [10]
small_sleep_times = [random.random() for _ in range(20)]
SLEEP_TIMES = big_sleep_times + small_sleep_times

# Launch remote tasks
refs = [sleep_task.remote(i) for i in SLEEP_TIMES]
for ref in refs:
    # Blocks until this ObjectRef is ready.
    result = ray.get(ref)  # Retrieve result in submission order.
    post_processing_step(result)  # Process the result.

In [ ]:
### YOUR CODE HERE ###

In [ ]:
if ray.is_initialized():
    ray.shutdown()

ray.init()

In [ ]:
### SAMPLE IMPLEMENTATION ###

# Launch remote tasks.
refs = [sleep_task.remote(i) for i in SLEEP_TIMES]
unfinished = refs
while unfinished:
    # Returns the first ObjectRef that is ready.
    finished, unfinished = ray.wait(unfinished, num_returns=1)
    # Retrieve the first ready result.
    result = ray.get(finished[0])
    # Process the result.
    post_processing_step(result)

In [ ]:
if ray.is_initialized():
    ray.shutdown()

ray.init()

In [ ]:
@ray.remote
def f(i: int) -> int:
    return i

# Anti-pattern: no parallelism due to calling ray.get inside of the loop.
sequential_returns = []
for i in range(100):
    sequential_returns.append(ray.get(f.remote(i)))

In [ ]:
### YOUR CODE HERE ###

In [ ]:
if ray.is_initialized():
    ray.shutdown()

ray.init()

In [ ]:
### SAMPLE IMPLEMENTATION ###
refs = []
for i in range(100):
    refs.append(f.remote(i))

parallel_returns = ray.get(refs)